In [1]:
from pony.orm import db_session
from be import db
from be.models import User, Course, Test, Question, Option, Tag, UserAnswer
import json

## Load seed data into db

In [2]:
db.bind(
    provider = 'mysql',
    host = 'localhost',
    user = 'its_admin',
    password = 'test123',
    db ='its-mysql'
)
db.generate_mapping(create_tables=True)

In [3]:
maths = Course(
    name='Maths'
)
geo = Course(
    name='Geography'
)
physics = Course(
    name='Physics'
)

def create_seed_data(file, course):
    with open(file, 'rb') as f:
        maths_questions = json.load(f)
        for q in maths_questions:
            tags = []
            for t in q['tags']:
                existing_tag = Tag.get(tag=t)
                if existing_tag:
                    tags.append(existing_tag)
                else:
                    tags.append(Tag(tag=t))
            question = Question(
                course=course, 
                question=q['question'],
                answer=q['answer'],
                difficulty='easy',
                tags=tags,
            )
            options = []
            for o in q['options']:
                options.append(Option(option=o,question=question))
            question.options = options
            
            
with db_session:
    create_seed_data('seed/maths.json', maths)
    create_seed_data('seed/geography.json', geo)
    create_seed_data('seed/physics.json', physics)

In [4]:
with db_session:
    if not User.get(username='ross'):
        User(username='ross', password='pbkdf2:sha256:600000$Ca8zFeQr4XQnlLxQ$2d0025b2dec190a3ffb9ed4964330d1a0497af45143d4b627042110e1716d101')

In [5]:
with db_session:
    user = User[1]
    course = Course[1]
    questions = Question.select().filter(course=1)
    available_questions = []
    for q in questions:
        if user not in q.tests.for_user:
            available_questions.append(q)
    print(available_questions)

[Question[1], Question[2], Question[3], Question[4], Question[5], Question[6], Question[7], Question[8], Question[9], Question[10], Question[11], Question[12], Question[13], Question[14], Question[15], Question[16], Question[17], Question[18], Question[19], Question[20], Question[21], Question[22], Question[23], Question[24]]
